In [79]:
import glob
import os
import re
import pandas as pd

In [80]:
# helps sort file by number at the end of each filename (10 correctly placed after 9 in filename)
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [81]:
# concatenate all csv files
ejmr_data = sorted(glob.glob('ejmr_pg*.csv'), key=numericalSort)

df_list = []
for filename in sorted(ejmr_data):
    df_list.append(pd.read_csv(filename))
full_df = pd.concat(df_list)

In [100]:
# split title into orig. link and post title, get freshness values
full_df['post_link'] = full_df['title'].str.extract(r'<a href="(.*?)">',expand=True)
full_df['post_title'] = full_df['title'].str.extract(r'">(.*?)</a>',expand=True)
full_df['how_fresh'] = full_df['freshness'].str.extract(r'">(.*?)</a>',expand=True)

list(full_df.columns)

# reorder columns
full_df = full_df[['post_title',
                   'num_posts',
                   'views',
                   'votes',
                   'how_fresh',
                   'download_time',
                   'title',
                   'post_link',
                   'freshness'
                  ]]

full_df.head()

# export full data to csv
# full_df.to_csv('ejmr_data.csv', index=True, index_label='pandas_index')

,post_title,num_posts,views,votes,how_fresh,download_time,title,post_link,freshness
0,Pls explain me how machines can learn?,5,95,0-0,1 day,2017-06-11 03:37:22.777761,"<a href=""https://www.econjobrumors.com/topic/p...",https://www.econjobrumors.com/topic/pls-explai...,"<a href=""https://www.econjobrumors.com/topic/p..."
1,Machine learning is Satan worshipping in disguise,17,337,6-0,1 day,2017-06-11 03:37:22.777852,"<a href=""https://www.econjobrumors.com/topic/m...",https://www.econjobrumors.com/topic/machine-le...,"<a href=""https://www.econjobrumors.com/topic/m..."
2,Should I buy a BMW,29,960,0-0,1 day,2017-06-11 03:37:22.777956,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/should-i-b...,"<a href=""https://www.econjobrumors.com/topic/s..."
3,A proposal on how to replace student evaluations,22,561,4-3,1 day,2017-06-11 03:37:22.778058,"<a href=""https://www.econjobrumors.com/topic/a...",https://www.econjobrumors.com/topic/a-proposal...,"<a href=""https://www.econjobrumors.com/topic/a..."
4,"Roman Catholics, put down your rosaries and ex...",1,69,1-0,1 day,2017-06-11 03:37:22.778133,"<a href=""https://www.econjobrumors.com/topic/r...",https://www.econjobrumors.com/topic/roman-cath...,"<a href=""https://www.econjobrumors.com/topic/r..."


In [ ]:
# NEXT?